<a href="https://colab.research.google.com/github/android-kunjapppan/PyTorch-Tutorial/blob/master/PyTorch_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Convolutional Neural Netoworks:
* Mainly work on image data and applys a operation called Convolutions
      Initial Image * Filter = Resulting Image
      (W-F+2P)/S + 1

* Max Pooling is used to down sample an Image by appplying maximum filter to sub-regions. REduces the size of the image. And tries to avoid overfitting.



In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

# Device Config

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyperparameters


In [3]:
num_epochs =4
batch_size = 4
learning_rate = 0.001

# Transforms:
* Dataset has PILImage images of range [0,1].
* We transform them to the Tensors of Normalized range [-1,1]

In [4]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

# Dataset:

In [5]:
train_dataset = torchvision.datasets.CIFAR10(root = './data',
                                             train = True,
                                             download = True,
                                             transform = transform
                                             )

test_dataset = torchvision.datasets.CIFAR10(root = './data',
                                             train = False,
                                             download = True,
                                             transform = transform
                                             )

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


# DataLoader

In [6]:
train_loader = torch.utils.data.DataLoader(train_dataset,
                                          batch_size = batch_size,
                                          shuffle = True
                                          )

test_loader = torch.utils.data.DataLoader(test_dataset,
                                          batch_size = batch_size,
                                          shuffle = False
                                          )

In [7]:
# Dataset Classes
classes = ('plane','car','bird','cat','deer',
           'dog','frog','horse','ship','truck')

# ConvNet Model

In [8]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)# 3 because the image has 3 channels 'RGB'
        # shape after this operation [4,6,28,28]

        self.pool = nn.MaxPool2d(2, 2)
        # [4,6,14,14]

        self.conv2 = nn.Conv2d(6, 16, 5)
        # [4,16,10,10]
    
        # will apply one more poolin layer in our network so the input size becomes [4,16,5,5]
    
        # 16*5*5 because the the o/p of previous layer will be [4,16,5,5] so will multiply 16*5*5
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # -> n, 3, 32, 32
        x = self.pool(F.relu(self.conv1(x)))  # -> n, 6, 14, 14
        x = self.pool(F.relu(self.conv2(x)))  # -> n, 16, 5, 5
        x = x.view(-1, 16 * 5 * 5)            # -> n, 400
        x = F.relu(self.fc1(x))               # -> n, 120
        x = F.relu(self.fc2(x))               # -> n, 84
        x = self.fc3(x)                       # -> n, 10
        return x

In [11]:
model = ConvNet().to(device)

In [12]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr = learning_rate)
n_total_steps = len(train_loader)

# Training:

In [13]:
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # origin shape: [4, 3, 32, 32] = 4, 3, 1024
        # input_layer: 3 input channels, 6 output channels, 5 kernel size
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 2000 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')


Epoch [1/4], Step [2000/12500], Loss: 2.2931
Epoch [1/4], Step [4000/12500], Loss: 2.3103
Epoch [1/4], Step [6000/12500], Loss: 2.3988
Epoch [1/4], Step [8000/12500], Loss: 2.0911
Epoch [1/4], Step [10000/12500], Loss: 1.4725
Epoch [1/4], Step [12000/12500], Loss: 2.6676
Epoch [2/4], Step [2000/12500], Loss: 2.2665
Epoch [2/4], Step [4000/12500], Loss: 1.8485
Epoch [2/4], Step [6000/12500], Loss: 1.5291
Epoch [2/4], Step [8000/12500], Loss: 1.7465
Epoch [2/4], Step [10000/12500], Loss: 2.2851
Epoch [2/4], Step [12000/12500], Loss: 1.4836
Epoch [3/4], Step [2000/12500], Loss: 1.1934
Epoch [3/4], Step [4000/12500], Loss: 1.5695
Epoch [3/4], Step [6000/12500], Loss: 2.5468
Epoch [3/4], Step [8000/12500], Loss: 0.9932
Epoch [3/4], Step [10000/12500], Loss: 1.7287
Epoch [3/4], Step [12000/12500], Loss: 1.3423
Epoch [4/4], Step [2000/12500], Loss: 3.1454
Epoch [4/4], Step [4000/12500], Loss: 2.3825
Epoch [4/4], Step [6000/12500], Loss: 2.0000
Epoch [4/4], Step [8000/12500], Loss: 1.3924
Epoc

In [14]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        # max returns (value ,index)
        _, predicted = torch.max(outputs, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()
        
        for i in range(batch_size):
            label = labels[i]
            pred = predicted[i]
            if (label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network: {acc} %')

    for i in range(10):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f'Accuracy of {classes[i]}: {acc} %')

Accuracy of the network: 47.47 %
Accuracy of plane: 59.2 %
Accuracy of car: 62.3 %
Accuracy of bird: 18.1 %
Accuracy of cat: 23.9 %
Accuracy of deer: 55.4 %
Accuracy of dog: 43.8 %
Accuracy of frog: 49.1 %
Accuracy of horse: 58.4 %
Accuracy of ship: 47.6 %
Accuracy of truck: 56.9 %
